# Prepare Dataset for Modeling

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import pickle
from sklearn.model_selection import train_test_split

## Import and Join Datasets

In [12]:
grid = pd.read_pickle("../../data/compacted_data/grade.pkl")
y = pd.read_pickle("../../data/compacted_data/y.pkl").rename(columns={"nuis": "y"})[["gid", "y"]]
df = grid.merge(y, on='gid', how='left').drop(["gid", "ID", "UF", "Row", "Col", "geometry"], axis=1)

In [13]:
y.loc[y["y"] != 0]

,gid,y
2239,100ME5233450N7998750,1.0
2260,100ME5233550N7998750,1.0
2261,100ME5233550N7998850,1.0
2279,100ME5233650N7998350,1.0
2280,100ME5233650N7998450,1.0
...,...,...
735045,100ME7090050N10419250,1.0
735053,100ME7090050N10420050,1.0
735054,100ME7090050N10420150,1.0
735078,100ME7090150N10420050,1.0


In [14]:
df.head()

,Polo,Declividade,Curvatura,APP30m,UCIntegral,AltaTensao,Vias50m,Dutovias,IndiceForma,DomSIden,...,t_vulner_mais1h,t_renda_trab,t_carteira_18m,t_scarteira_18m,t_setorpublico_18m,t_contapropria_18m,t_empregador_18m,t_formal_18m,t_atividade10a15,y
0,Porto Alegre,4.919002,6.333333,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,Porto Alegre,4.900816,1.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Porto Alegre,4.152381,4.666667,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Porto Alegre,3.205212,7.777778,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,Porto Alegre,5.218916,4.166667,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


## Handle missing values

In [15]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing.sort_values(ascending=True)
drop_cols = percent_missing.loc[percent_missing > 50].index.tolist()
df.drop(drop_cols, axis = 1, inplace=True)

In [23]:
len(drop_cols)
len(df.columns)

72

In [19]:
df

,Polo,Declividade,Curvatura,APP30m,UCIntegral,AltaTensao,Vias50m,Dutovias,IndiceForma,DomSIden,...,NDenDom,NDenPop,NMoradores,NPes10Alf,NRespAlf,NRespFem,NRespIdade,NResp30,NResp30NAlf,y
0,Porto Alegre,4.919002,6.333333,0.000000,0.0,0.0,1.0,0.0,0.000000,0.000000,...,0.000000,0.000000,3.35,85.349462,82.899628,21.189591,47.122677,0.828996,0.838565,0.0
1,Porto Alegre,4.900816,1.000000,0.000000,0.0,0.0,1.0,0.0,0.000000,0.000000,...,0.000000,0.000000,3.35,85.349462,82.899628,21.189591,47.122677,0.825195,0.834720,0.0
2,Porto Alegre,4.152381,4.666667,0.333333,0.0,0.0,1.0,0.0,0.000000,0.000000,...,0.000000,0.000000,3.35,85.349462,82.899628,21.189591,47.122677,0.527855,0.533947,0.0
3,Porto Alegre,3.205212,7.777778,0.000000,0.0,0.0,1.0,0.0,0.000000,0.000000,...,0.000000,0.000000,3.35,85.349462,82.899628,21.189591,47.122677,0.828996,0.838565,0.0
4,Porto Alegre,5.218916,4.166667,0.000000,0.0,0.0,1.0,0.0,0.000000,0.000000,...,8213.489664,27480.076944,3.35,85.349462,82.899628,21.189591,47.122677,0.828996,0.838565,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735153,Recife,1.944661,3.511628,0.000000,0.0,0.0,1.0,0.0,0.938342,82.352941,...,64.516130,178.368120,2.76,73.684211,58.823529,0.000000,47.941176,0.344267,0.585254,0.0
735154,Recife,3.191298,3.777778,0.000000,0.0,0.0,1.0,0.0,0.491669,82.352941,...,64.516130,178.368120,2.76,73.684211,58.823529,0.000000,47.941176,0.246509,0.419066,0.0
735155,Recife,1.246800,1.888889,0.000000,0.0,0.0,1.0,0.0,0.000000,82.352941,...,64.516130,178.368120,2.76,73.684211,58.823529,0.000000,47.941176,0.250660,0.426122,0.0
735156,Recife,1.599806,1.418605,0.000000,0.0,0.0,1.0,0.0,0.000000,82.352941,...,64.516130,178.368120,2.76,73.684211,58.823529,0.000000,47.941176,0.177649,0.302003,0.0


In [24]:
result = df.groupby('Polo')['y'].mean() * 100
result

Polo
Belo Horizonte       22.250405
Brasília             10.891307
Juazeiro do Norte    21.356532
Marabá               27.324308
Porto Alegre          7.455546
Recife               24.956744
Name: y, dtype: float64

In [11]:
df = df.fillna(df.median())

TypeError: Cannot convert [['Porto Alegre' 'Porto Alegre' 'Porto Alegre' ... 'Recife' 'Recife'
  'Recife']] to numeric

In [ ]:
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing.sort_values(ascending=True)

Polo            0.0
LixoRio         0.0
LixoJogado      0.0
LixoAterrado    0.0
LixoQueimado    0.0
               ... 
DomCLixAc       0.0
DomSRedeEsg     0.0
DomSEsg         0.0
NResp30NAlf     0.0
y               0.0
Length: 72, dtype: float64

## Split training and test

In [7]:
polos = ['Porto Alegre', 'Marabá', 'Brasília', 'Belo Horizonte', 'Juazeiro do Norte', 'Recife']

In [8]:
for p in polos:
    test = df[df.iloc[:, 0] == p]
    X_test = test.loc[:, test.columns != "y"]
    y_test = test["y"]
    X_test.to_pickle(f"../../data/model_input/X_{p}.pkl")
    y_test.to_pickle(f"../../data/model_input/y_{p}.pkl")